In [ ]:
import logging
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import SimpleITK as sitk

from radiomics import featureextractor, imageoperations

In [ ]:
root_dir = pathlib.Path("..").resolve()

In [ ]:
def show_image_and_masks(img_path, mask_path):
    if isinstance(img_path, pathlib.Path) or isinstance(img_path, str):
        image = sitk.ReadImage(img_path)
    else:
        image = img_path
    if isinstance(mask_path, pathlib.Path) or isinstance(mask_path, str):
        mask = sitk.ReadImage(mask_path)
    else:
        mask = mask_path
    image_data = sitk.GetArrayFromImage(image).T
    mask_data = sitk.GetArrayFromImage(mask).T

    center_slice = image_data.shape[2] // 2
    unique_labels = np.unique(mask_data.ravel())

    _, axs = plt.subplots(1, 3, figsize=(12, 12))
    for i, slice in enumerate(range(center_slice - 1, center_slice + 2)):
        axs[i].imshow(image_data[:, :, slice], cmap="gray")
        im = axs[i].imshow(
            mask_data[:, :, slice],
            cmap="jet",
            alpha=np.where(mask_data[:, :, slice] == 0, 0, 0.3),
        )
        axs[i].grid(False)
        axs[i].axis("off")
    colors = [im.cmap(im.norm(value)) for value in unique_labels]
    patches = [
        mpatches.Patch(color=colors[i], label=f"{unique_labels[i]}")
        for i in range(len(unique_labels))
    ]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
    plt.tight_layout()
    plt.show()


def checkMaskVol(image, mask, label):
    try:
        imageoperations.checkMask(
            image, mask, minimumROIDimensions=3, minimumROISize=1000, label=label
        )
        result = label
    except Exception as e:
        result = None
    return result

In [ ]:
filtered_midas_img_relation = pd.read_csv(
    root_dir.joinpath("data", "filtered_midas900_t2w.csv"), sep=","
)
filtered_midas_img_relation["Subject_MIDS"] = filtered_midas_img_relation["Image"].map(
    lambda x: x.split("/")[8]
)
filtered_midas_img_relation["Session_MIDS"] = filtered_midas_img_relation["Image"].map(
    lambda x: x.split("/")[9]
)
filtered_midas_img_relation["Subject_XNAT"] = filtered_midas_img_relation[
    "Subject_MIDS"
].map(lambda x: f"ceibcs_S{int(x.split('sub-S')[1])}")
filtered_midas_img_relation["Session_XNAT"] = filtered_midas_img_relation[
    "Session_MIDS"
].map(lambda x: f"ceibcs_E{int(x.split('ses-E')[1])}")
filtered_midas_img_relation

In [ ]:
index = 1
show_image_and_masks(
    filtered_midas_img_relation.iloc[index]["Image"],
    filtered_midas_img_relation.iloc[index]["Mask"],
)

In [ ]:
%%writefile ../src/Params.yaml
setting:
  binWidth: 25
  correctMask: True
  interpolator: 'sitkBSpline' # This is an enumerated value, here None is not allowed
  normalize: True
  resampledPixelSpacing: # This disables resampling, as it is interpreted as None, to enable it, specify spacing in x, y, z as [x, y , z]
  weightingNorm: # If no value is specified, it is interpreted as None

# Image types to use: "Original" for unfiltered image, for possible filters, see documentation.
imageType:
  Original: {} # for dictionaries / mappings, None values are not allowed, '{}' is interpreted as an empty dictionary
  LoG:
   sigma: [1.0, 3.0, 5.0]
  Wavelet: {}

featureClass:
  firstorder:
  glcm:
  gldm:
  glrlm: 
  glszm:
  shape:

In [ ]:
params = root_dir.joinpath("src", "Params.yaml")

In [ ]:
def run(case, how="3d"):
    if params.exists():
        extractor = featureextractor.RadiomicsFeatureExtractor(str(params))
    else:  # Parameter file not found, use hardcoded settings instead
        settings = {}
        settings["binWidth"] = 25
        settings["resampledPixelSpacing"] = None
        settings["interpolator"] = sitk.sitkBSpline
        settings["enableCExtensions"] = True
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    logging.info(
        "Processing Patient %s (Image: %s, Mask: %s)",
        case["ID"],
        case["Image"],
        case["Mask"],
    )

    image_path = case["Image"]
    mask_path = case["Mask"]
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)
    labels = np.unique(sitk.GetArrayFromImage(mask).ravel())
    valid_labels = []
    for label in labels:
        result = checkMaskVol(image, mask, label)
        if result:
            valid_labels.append(result)

    if how == "center slice":
        center_slice = image.GetSize()[2] // 2
        image = image[:, :, center_slice]
        mask = mask[:, :, center_slice]

    patient = []
    for index, label in enumerate(valid_labels[:5], start=1):
        label = int(label)
        logging.info(
            "Processing Patient %s (Image: %s, Mask: %s, Label: %s)",
            case["ID"],
            case["Image"],
            case["Mask"],
            label,
        )
        if (image_path is not None) and (mask_path is not None):
            try:
                result = pd.Series(extractor.execute(image, mask, label))
            except Exception:
                logging.error("FEATURE EXTRACTION FAILED:", exc_info=True)
                result = pd.Series()
        else:
            logging.error("FEATURE EXTRACTION FAILED: Missing Image and/or Mask")
            result = pd.Series()

        result.name = case["ID"]
        result = result.add_prefix("label{}_".format(index))
        patient.append(result)
    return patient

In [ ]:
run(filtered_midas_img_relation.iloc[index], how="center slice")

## Mask perturbation

In [ ]:
image_path = filtered_midas_img_relation.iloc[index]["Image"]
mask_path = filtered_midas_img_relation.iloc[index]["Mask"]

In [ ]:
mask = sitk.ReadImage(mask_path)

In [ ]:
def random_shift(mask, max_shift_x=3, max_shift_y=5):
    dx = np.random.randint(-max_shift_x, max_shift_x + 1)
    dy = np.random.randint(-max_shift_y, max_shift_y + 1)
    print(dx, dy)
    translation = sitk.TranslationTransform(3, [dx, dy, 0])
    shifted_mask = sitk.Resample(mask, translation)
    return shifted_mask

In [ ]:
mask_shift = random_shift(mask)
show_image_and_masks(
    image_path,
    mask_shift,
)

In [ ]:
def erosion(mask):
    eroded_mask = sitk.Image(mask)
    for i in np.unique(sitk.GetArrayFromImage(mask).astype(np.double)):
        if i != 0:
            eroded_mask = sitk.BinaryErode(eroded_mask, foregroundValue=i)
    return eroded_mask

In [ ]:
mask_erosion = erosion(mask)
show_image_and_masks(
    image_path,
    mask_erosion,
)

In [ ]:
def dilation(mask):
    dilated_mask = sitk.Image(mask)
    for i in np.unique(sitk.GetArrayFromImage(mask).astype(np.double)):
        if i != 0:
            dilated_mask = sitk.BinaryDilate(dilated_mask, foregroundValue=i)
    return dilated_mask

In [ ]:
mask_dilation = dilation(mask)
show_image_and_masks(
    image_path,
    mask_dilation,
)